# 本例程是利用lstm搭建的recurrent neural network来对imdb电影评论数据进行分类

In [1]:
from __future__ import division
from __future__ import print_function

In [2]:
import tensorflow as tf
import numpy as np

In [4]:
# max total num of words
max_features = 5000

In [5]:
from utils import imdb
from utils import sequence

In [7]:
print('loading data')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

loading data
25000 train sequences
25000 test sequences


In [16]:
seq_len = map(lambda x: len(x), X_train)

In [19]:
print(seq_len[0])

218


In [18]:
print(len(X_train[0]))
print(len(X_train[1]))

218
189


In [6]:
print(y_train.shape)

(25000,)


In [7]:
maxlen = 400
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [8]:
# left padding with right align
print(X_train.shape)
print(X_train[0])

(25000, 400)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    1   14   22   16   43  530  973 1622 1385   65  458 4468   66

In [9]:
# build the graph functions
# add the placeholders
def add_placeholders():
    input_placeholder = tf.placeholder(tf.int32, shape=[batch_size, num_steps])
    label_placeholder = tf.placeholder(tf.float32, shape=[batch_size])
    
    return input_placeholder, label_placeholder

In [10]:
# create the feed_dict
def create_feed_dict(input_placeholder, input_batch, label_placeholder, label_batch):
    feed_dict = {input_placeholder: input_batch,
                label_placeholder:label_batch}
    return feed_dict

In [29]:
def add_embed_layer(vocab_size, input_placeholder):
    with tf.device('/cpu:0'):
        embed = tf.get_variable(name="Embedding", shape=[vocab_size, embed_size])
        inputs = tf.nn.embedding_lookup(embed, input_placeholder)
        #inputs = [tf.squeeze(input, squeeze_dims=[1]) for input in tf.split(1, num_steps, inputs)]
        return inputs

In [12]:
## add training op
def add_train_op(loss):
    train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
    return train_op

In [13]:
## add rnn model
def add_rnn_model(hidden_size, num_steps):
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=0.0)
    #cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell]*num_steps)
    cell = lstm_cell
    return cell

In [14]:
# evalate the prediction 
def evaluation(y_pred_sigmoid, label_placeholder):
    y_pred_label = (y_pred_sigmoid > 0.5)
    label_placeholder = tf.cast(label_placeholder, tf.bool)
    correct_pred_num = []
    correct_pred_num.append(tf.reduce_sum(tf.cast(tf.equal(y_pred_label, label_placeholder), tf.int32)))
    correct_pred_num = np.sum(correct_pred_num)
    return correct_pred_num

In [72]:
def do_evaluation(sess, X, y):
    total_correct_num = 0
    num_steps = len(X) // batch_size
    for step in range(num_steps):
        # generate the data feed dict
        input_batch = X[step*batch_size:(step+1)*batch_size, :]
        label_batch = y[step*batch_size:(step+1)*batch_size]

        feed = {input_placeholder:input_batch, label_placeholder:label_batch }
        correct_num_step = sess.run([correct_num], feed)
        total_correct_num += correct_num_step[0]
    print('Testing Accuracy: %f' %(total_correct_num/(num_steps*batch_size)))

In [73]:
#when debug set max_epochs = 1
max_epochs = 15
batch_size = 20
hidden_size = 100
num_steps = maxlen
embed_size = 50

In [74]:
# majority classfier
majority = np.sum(y_train) / len(X_train)
if majority > 0.5:
    print('the majority classifier accuracy is %f' %(majority))
else:
    print('the majority classifier accuracy is %f' %(1-majority))

the majority classifier accuracy is 0.500000


In [75]:
print(len(X_train))

print(np.sum(y_train))

25000
12500


In [76]:
def conv1d(x, W):
    return tf.nn.conv1d(x, W, stride=1, padding="SAME")

In [77]:
help(tf.nn.conv1d)

Help on function conv1d in module tensorflow.python.ops.nn_ops:

conv1d(value, filters, stride, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
    Computes a 1-D convolution given 3-D input and filter tensors.
    
    Given an input tensor of shape [batch, in_width, in_channels]
    and a filter / kernel tensor of shape
    [filter_width, in_channels, out_channels], this op reshapes
    the arguments to pass them to conv2d to perform the equivalent
    convolution operation.
    
    Internally, this op reshapes the input tensors and invokes
    `tf.nn.conv2d`.  A tensor of shape [batch, in_width, in_channels]
    is reshaped to [batch, 1, in_width, in_channels], and the filter
    is reshaped to [1, filter_width, in_channels, out_channels].
    The result is then reshaped back to [batch, out_width, out_channels]
    (where out_width is a function of the stride and padding as in
    conv2d) and returned to the caller.
    
    Args:
      value: A 3D `Tensor`.  Must be o

In [78]:
## the tf lib not contain the max_pool1d,
## we can add one dummy dimension in the input tensor
## x should be [batch_size, height, width, channels]
## x : [batch_size, height, channels]
## => [batch_size, height, channels, 1]
## => [batch_size, height, 1, channels]
def max_pool1d(x):
    x = tf.expand_dims(x, -1)
    x = tf.transpose(x,perm=[0,1,3,2]) ##
    ## invoke maxpool
    pool_out = tf.nn.max_pool(x, ksize=[1,2,1,1], strides=[1,4,1,1], padding="SAME")
    #pool_out = tf.transpose(pool_out,perm=[0,2,1,3])
    pool_out = tf.squeeze(x, squeeze_dims=[2])
    return pool_out

In [9]:
help(tf.nn.rnn_cell)

Help on module tensorflow.python.ops.rnn_cell in tensorflow.python.ops:

NAME
    tensorflow.python.ops.rnn_cell - Module for constructing RNN Cells.

FILE
    /home/wang/tensorflow_dl/local/lib/python2.7/site-packages/tensorflow/python/ops/rnn_cell.py

DESCRIPTION
    ## Base interface for all RNN Cells
    
    @@RNNCell
    
    ## RNN Cells for use with TensorFlow's core RNN methods
    
    @@BasicRNNCell
    @@BasicLSTMCell
    @@GRUCell
    @@LSTMCell
    
    ## Classes storing split `RNNCell` state
    
    @@LSTMStateTuple
    
    ## RNN Cell wrappers (RNNCells that wrap other RNNCells)
    
    @@MultiRNNCell
    @@DropoutWrapper
    @@EmbeddingWrapper
    @@InputProjectionWrapper
    @@OutputProjectionWrapper

CLASSES
    __builtin__.object
        RNNCell
            BasicLSTMCell
            BasicRNNCell
            DropoutWrapper
            EmbeddingWrapper
            GRUCell
            InputProjectionWrapper
            LSTMCell
            MultiRNNCell
            

In [13]:
help(tf.nn.dynamic_rnn)

Help on function dynamic_rnn in module tensorflow.python.ops.rnn:

dynamic_rnn(cell, inputs, sequence_length=None, initial_state=None, dtype=None, parallel_iterations=None, swap_memory=False, time_major=False, scope=None)
    Creates a recurrent neural network specified by RNNCell `cell`.
    
    This function is functionally identical to the function `rnn` above, but
    performs fully dynamic unrolling of `inputs`.
    
    Unlike `rnn`, the input `inputs` is not a Python list of `Tensors`, one for
    each frame.  Instead, `inputs` may be a single `Tensor` where
    the maximum time is either the first or second dimension (see the parameter
    `time_major`).  Alternatively, it may be a (possibly nested) tuple of
    Tensors, each of them having matching batch and time dimensions.
    The corresponding output is either a single `Tensor` having the same number
    of time steps and batch size, or a (possibly nested) tuple of such tensors,
    matching the nested structure of `cell.o

## CNN Model

In [16]:
help(np.concatenate)

Help on built-in function concatenate in module numpy.core.multiarray:

concatenate(...)
    concatenate((a1, a2, ...), axis=0)
    
    Join a sequence of arrays along an existing axis.
    
    Parameters
    ----------
    a1, a2, ... : sequence of array_like
        The arrays must have the same shape, except in the dimension
        corresponding to `axis` (the first, by default).
    axis : int, optional
        The axis along which the arrays will be joined.  Default is 0.
    
    Returns
    -------
    res : ndarray
        The concatenated array.
    
    See Also
    --------
    ma.concatenate : Concatenate function that preserves input masks.
    array_split : Split an array into multiple sub-arrays of equal or
                  near-equal size.
    split : Split array into a list of multiple sub-arrays of equal size.
    hsplit : Split array into multiple sub-arrays horizontally (column wise)
    vsplit : Split array into multiple sub-arrays vertically (row wise)
    dsp

In [20]:
print(np.ones([2]))

[ 1.  1.]


In [80]:
with tf.Graph().as_default(), tf.Session() as sess:
    input_placeholder, label_placeholder = add_placeholders()
    vocab_size = max_features
    #Embed layer
    inputs = add_embed_layer(vocab_size, input_placeholder)
    #CNN layer
    W_cnn = tf.Variable(initial_value =tf.truncated_normal(shape=[5,50, 64], stddev=0.1))
    b_cnn = tf.Variable(initial_value=tf.truncated_normal(shape=[64], stddev=0.1))
    cnn1_out = tf.nn.relu(conv1d(inputs, W_cnn) + b_cnn)
    #Pooling layer
    #pool_out: [batch_size, height/2, channels]
    
    pool_out = max_pool1d(cnn1_out)
    pool_out_flat = tf.reshape(pool_out, [-1, 400*64])
    
    ##Fully connected layer
    W_fc = tf.Variable(initial_value=tf.truncated_normal(shape=[400*64, 100], stddev=0.1))
    b_fc = tf.Variable(initial_value=tf.truncated_normal(shape=[100], stddev=0.1))
    fc_out = tf.nn.relu(tf.matmul(pool_out_flat, W_fc) + b_fc)    
    ##sigmoid layer
    W_classifier = tf.Variable(initial_value=tf.truncated_normal(shape=[100, 1], stddev=0.1))
    b_classifier = tf.Variable(initial_value=tf.truncated_normal(shape=[1], stddev=0.1))
    ##
    y_pred = tf.squeeze(tf.matmul(fc_out, W_classifier) + b_classifier)
    y_pred_sigmoid = tf.nn.sigmoid(y_pred)
    ##
    correct_num = evaluation(y_pred_sigmoid, label_placeholder)
    
    loss = tf.nn.sigmoid_cross_entropy_with_logits(y_pred, label_placeholder)
    
    train_op = add_train_op(loss)
    
    #state_step = initial_state
    sess.run(tf.initialize_all_variables())
    
    for epoch in range(max_epochs):
        print('%d Epoch starts, Training....' %(epoch))
        mean_loss = []
        total_correct_num = 0
        for step in range(len(X_train) // batch_size):

            input_batch = X_train[step*batch_size:(step+1)*batch_size, :]
            label_batch = y_train[step*batch_size:(step+1)*batch_size]
            
           
            
            feed = {input_placeholder:input_batch, label_placeholder:label_batch}
            '''
            inputs_step, cnn1_out_step, pool_out_step = sess.run([inputs, cnn1_out, pool_out], feed)
            print(inputs_step.shape)
            print(cnn1_out_step.shape)
            print(pool_out_step.shape)
            '''
            _, y_pred_step, loss_step, correct_num_step= sess.run([train_op, y_pred, loss, correct_num], feed)
            
            loss_step = np.sum(loss_step)
            mean_loss.append(loss_step)
            total_correct_num += correct_num_step
            
            if step % 100 == 0:
                print('step %d / %d : loss : %f' %(step, len(X_train) // batch_size, np.mean(mean_loss)))
                mean_loss = []
            
        print('precision: %f' %(total_correct_num/len(X_train)))
        print('Testing....')
        do_evaluation(sess, X_test, y_test)
   

0 Epoch starts, Training....
step 0 / 1250 : loss : 13.765029
step 100 / 1250 : loss : 14.484235
step 200 / 1250 : loss : 13.847551
step 300 / 1250 : loss : 10.450971
step 400 / 1250 : loss : 7.594388
step 500 / 1250 : loss : 6.681324
step 600 / 1250 : loss : 6.479756
step 700 / 1250 : loss : 6.201917
step 800 / 1250 : loss : 6.016019
step 900 / 1250 : loss : 5.446662
step 1000 / 1250 : loss : 5.430323
step 1100 / 1250 : loss : 5.432162
step 1200 / 1250 : loss : 5.328698
precision: 0.803800
Testing....
Testing Accuracy: 0.891960
1 Epoch starts, Training....
step 0 / 1250 : loss : 1.574687
step 100 / 1250 : loss : 4.840659
step 200 / 1250 : loss : 5.337465
step 300 / 1250 : loss : 4.002223
step 400 / 1250 : loss : 3.377989
step 500 / 1250 : loss : 3.286219
step 600 / 1250 : loss : 3.347501
step 700 / 1250 : loss : 3.561467
step 800 / 1250 : loss : 3.528086
step 900 / 1250 : loss : 3.089107
step 1000 / 1250 : loss : 2.804906
step 1100 / 1250 : loss : 2.924674
step 1200 / 1250 : loss : 2.

KeyboardInterrupt: 

In [66]:
help(tf.nn.dropout)

Help on function dropout in module tensorflow.python.ops.nn_ops:

dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
    Computes dropout.
    
    With probability `keep_prob`, outputs the input element scaled up by
    `1 / keep_prob`, otherwise outputs `0`.  The scaling is so that the expected
    sum is unchanged.
    
    By default, each element is kept or dropped independently.  If `noise_shape`
    is specified, it must be
    [broadcastable](http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)
    to the shape of `x`, and only dimensions with `noise_shape[i] == shape(x)[i]`
    will make independent decisions.  For example, if `shape(x) = [k, l, m, n]`
    and `noise_shape = [k, 1, 1, n]`, each batch and channel component will be
    kept independently and each row and column will be kept or not kept together.
    
    Args:
      x: A tensor.
      keep_prob: A scalar `Tensor` with the same type as x. The probability
        that each element is kept.
   

## RNN Model

In [151]:
with tf.Graph().as_default(), tf.Session() as sess:
    input_placeholder, label_placeholder = add_placeholders()
    initial_state_placeholder = tf.placeholder(tf.float32)
    vocab_size = max_features
    inputs = add_embed_layer(vocab_size, input_placeholder)
    
    ##initial state
    cell = add_rnn_model(hidden_size, num_steps)
    initial_state = cell.zero_state(batch_size, tf.float32)

    # state is the final state
    outputs, state = tf.nn.rnn(cell, inputs, initial_state=initial_state)
    #add projection layer
    W = tf.get_variable('Weights', shape=[hidden_size, 1])
    b = tf.get_variable('Bias', shape = [1])
    
    y_pred = tf.squeeze(tf.matmul(outputs[-1], W) )
    
    y_pred_sigmoid = tf.sigmoid(y_pred)
    
    correct_num = evaluation(y_pred_sigmoid, label_placeholder)
    
    loss = tf.nn.sigmoid_cross_entropy_with_logits(y_pred, label_placeholder)
    
    train_op = add_train_op(loss)
    
    #state_step = initial_state
    sess.run(tf.initialize_all_variables())

    for epoch in range(max_epochs):
        print('%d Epoch starts, Training....' %(epoch))
        mean_loss = []
        total_correct_num = 0
        for step in range(len(X_train) // batch_size):
            # generate the data feed dict
            if step == 0:
                init_state = sess.run([initial_state])
            else:
                init_state = state_step[-1]
            input_batch = X_train[step*batch_size:(step+1)*batch_size, :]
            label_batch = y_train[step*batch_size:(step+1)*batch_size]
            
            '''
            feed = create_feed_dict(input_placeholder, input_batch, 
                                    label_placeholder, label_batch, 
                                    initial_state_placeholder, initial_state)
            '''
            feed = {input_placeholder:input_batch, label_placeholder:label_batch, initial_state_placeholder:init_state }
            _, state_step , y_pred_step, loss_step, correct_num_step= sess.run([train_op, state, y_pred, loss, correct_num], feed)
            
            loss_step = np.sum(loss_step)
            mean_loss.append(loss_step)
            total_correct_num += correct_num_step
            
            if step % 100 == 0:
                print('step %d / %d : loss : %f' %(step, len(X_train) // batch_size, np.mean(mean_loss)))
                mean_loss = []
            do_evaluation(sess, X_test, y_test)
        print('precision: %f' %(total_correct_num/len(X_train)))
        print('Testing....')
        do_evaluation(sess, X_test, y_test)

KeyboardInterrupt: 

In [152]:
help(tf.nn.sigmoid_cross_entropy_with_logits)

Help on function sigmoid_cross_entropy_with_logits in module tensorflow.python.ops.nn:

sigmoid_cross_entropy_with_logits(logits, targets, name=None)
    Computes sigmoid cross entropy given `logits`.
    
    Measures the probability error in discrete classification tasks in which each
    class is independent and not mutually exclusive.  For instance, one could
    perform multilabel classification where a picture can contain both an elephant
    and a dog at the same time.
    
    For brevity, let `x = logits`, `z = targets`.  The logistic loss is
    
          z * -log(sigmoid(x)) + (1 - z) * -log(1 - sigmoid(x))
        = z * -log(1 / (1 + exp(-x))) + (1 - z) * -log(exp(-x) / (1 + exp(-x)))
        = z * log(1 + exp(-x)) + (1 - z) * (-log(exp(-x)) + log(1 + exp(-x)))
        = z * log(1 + exp(-x)) + (1 - z) * (x + log(1 + exp(-x))
        = (1 - z) * x + log(1 + exp(-x))
        = x - x * z + log(1 + exp(-x))
    
    For x < 0, to avoid overflow in exp(-x), we reformulate the a

In [ ]:
help(tf.nn.rnn_cell.BasicLSTMCell)

In [ ]:
help(tf.shape)

In [ ]:
tf.reset_default_graph()

# Create input data
#X = np.random.randn(2, 10, 8)
X= tf.random_uniform(dtype=tf.float,shape=[2,10,8])
# The second example is of length 6 
#X[1,6,:] = 0
#X_lengths = [10, 6]
X_new = []
for index in range(8):
    X_new.append(X[:,:,index])
#X_new = (tf.squeeze(X[0,:,:]),tf.squeeze(X[1,:,:]))
cell = tf.nn.rnn_cell.LSTMCell(num_units=64, state_is_tuple=True)
#cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * 10)

outputs, last_states = tf.nn.rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X_new)

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)

assert result[0]["outputs"].shape == (2, 10, 64)
print(result[0]["outputs"])

# Outputs for the second example past past length 6 should be 0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()